# Get Most relevant News for S&P 500

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ast

In [2]:
# expand the column width to see the full sentences
pd.set_option('display.max_colwidth', None)

In [3]:
nyt_df = pd.read_csv('../Data/Original/nyt-metadata.csv')
nyt_df.head()

/var/folders/gt/_n6gk0zn66v7tnqb9xpcyk_c0000gn/T/ipykernel_88429/511978975.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  nyt_df = pd.read_csv('../Data/Original/nyt-metadata.csv')


,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,subsection_name
0,Article on upcoming New York Giants-Dallas Cowboys game; photo (M),https://www.nytimes.com/2000/01/01/sports/pro-football-playoffs-or-no-dallas-provides-the-motivation.html,Article on upcoming New York Giants-Dallas Cowboys game; photo (M),"Waiting in the visiting locker room at Texas Stadium late tomorrow afternoon, the Giants will know whether the Green Bay Packers, who play earlier against Arizona, have won or are comfortably ahead.",D,2.0,The New York Times,[],"{'main': 'Playoffs or No, Dallas Provides The Motivation', 'kicker': 'PRO FOOTBALL', 'content_kicker': None, 'print_headline': 'PRO FOOTBALL; Playoffs or No, Dallas Provides The Motivation', 'name': None, 'seo': None, 'sub': None}","[{'name': 'organizations', 'value': 'New York Giants', 'rank': 1, 'major': 'N'}, {'name': 'organizations', 'value': 'Dallas Cowboys', 'rank': 2, 'major': 'N'}, {'name': 'subject', 'value': 'Football', 'rank': 3, 'major': 'N'}]",2000-01-01 05:00:00+00:00,article,Sports Desk,Sports,"{'original': 'By Bill Pennington', 'person': [{'firstname': 'Bill', 'middlename': None, 'lastname': 'Pennington', 'qualifier': None, 'title': None, 'role': 'reported', 'organization': '', 'rank': 1}], 'organization': None}",News,nyt://article/01111a48-3502-5021-8096-bc9293797d54,819.0,nyt://article/01111a48-3502-5021-8096-bc9293797d54,NaN
1,"Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing",https://www.nytimes.com/2000/01/01/opinion/l-on-this-first-day-a-fanfare-for-the-new-era-knowing-the-world-165590.html,"Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing",To the Editor:,A,30.0,The New York Times,[],"{'main': 'On This First Day, a Fanfare for the New Era; Knowing the World', 'kicker': None, 'content_kicker': None, 'print_headline': 'On This First Day, a Fanfare for the New Era; Knowing the World', 'name': None, 'seo': None, 'sub': None}","[{'name': 'persons', 'value': 'Pond, Jeanne C', 'rank': 1, 'major': 'N'}, {'name': 'subject', 'value': 'TWO THOUSAND (YEAR)', 'rank': 2, 'major': 'N'}]",2000-01-01 05:00:00+00:00,article,Editorial Desk,Opinion,"{'original': '', 'person': [], 'organization': None}",Letter,nyt://article/02328edc-dad1-5eb0-900e-917162e46dcd,122.0,nyt://article/02328edc-dad1-5eb0-900e-917162e46dcd,NaN
2,"Many experts on Y2K computer problem report that Internet performed impressively through rollover from 1999 to 2000 even if it did sag at times in isolated sites as users turned to e-mail messages, Web sites, newsgroups and electronic chat rooms to track arrival of Year 2000 in more and more time zones (M)",https://www.nytimes.com/2000/01/01/us/1-1-00-technology-2000-momentous-relief-internet-s-cheering-squad-nervously.html,"Many experts on Y2K computer problem report that Internet performed impressively through rollover from 1999 to 2000 even if it did sag at times in isolated sites as users turned to e-mail messages, Web sites, newsgroups and electronic chat rooms t...","As the world slid nervously yesterday through the shadow of its first global technology challenge, many year 2000 experts took special comfort in the relatively stable behavior of one of their principal tools in dealing with the problem: the not-always-reliable Internet.",A,10.0,The New York Times,[],"{'main': ""Internet's Cheering Squad Nervously Watches Clock"", 'kicker': '1/1/00: TECHNOLOGY AND 2000 -- MOMENTOUS RELIEF', 'content_kicker': None, 'print_headline': ""1/1/00: TECHNOLOGY AND 2000 -- MOMENTOUS RELIEF; Internet's Cheering Squad Nervously Watches Clock"", 'name': None, 'seo': None, 'sub': None}","[{'name': 'subject', 'value': 'Electronic Mail', 'rank': 1, 'major': 'N'}, {'name

In [5]:
nyt_df.shape

(2191867, 20)

In [4]:
nyt_df['section_name'].unique()

array(['Sports', 'Opinion', 'U.S.', 'Archives', 'New York',
       'Business Day', 'Technology', 'Arts', 'Books', 'World', 'Theater',
       'Education', 'Movies', 'Travel', 'Style', 'Real Estate',
       'Magazine', 'Week in Review', 'Automobiles', 'Fashion & Style',
       'Health', 'Science', 'Job Market', 'Food', 'Home & Garden',
       'T Magazine', 'Giving', 'Obituaries', 'The Learning Network',
       'membercenter', nan, 'Climate', 'Your Money', 'Corrections',
       'Homepage', 'readersopinions', 'Crosswords & Games',
       'Editors Picks', 'Great Homes & Destinations', 'nationchallenged',
       'Multimedia/Photos', "Critic's Choice", 'College', 'timestalks',
       'Washington', 'none', 'timesselect', 'Blogs', 'Times Topics',
       'Well', 'nytfrontpage', 'UrbanEye', 'xword', 'Sunday Review',
       'Global Home', 'Public Editor', 'Booming', 'The Upshot', 'Feeds',
       'International Home', 'Times Insider', 'NYT Now', 'Admin',
       'Universal', 'Topics', 'Today’s Paper

In [8]:
clean_nyt_df = pd.DataFrame()

clean_nyt_df['date'] = nyt_df['pub_date']
clean_nyt_df['abstract'] = nyt_df['abstract']
clean_nyt_df['lead_paragraph'] = nyt_df['lead_paragraph']
clean_nyt_df['headline'] = nyt_df['headline']
clean_nyt_df['section_name'] = nyt_df['section_name']

clean_nyt_df.head()

,date,abstract,lead_paragraph,headline,section_name
0,2000-01-01 05:00:00+00:00,Article on upcoming New York Giants-Dallas Cowboys game; photo (M),"Waiting in the visiting locker room at Texas Stadium late tomorrow afternoon, the Giants will know whether the Green Bay Packers, who play earlier against Arizona, have won or are comfortably ahead.","{'main': 'Playoffs or No, Dallas Provides The Motivation', 'kicker': 'PRO FOOTBALL', 'content_kicker': None, 'print_headline': 'PRO FOOTBALL; Playoffs or No, Dallas Provides The Motivation', 'name': None, 'seo': None, 'sub': None}",Sports
1,2000-01-01 05:00:00+00:00,"Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing",To the Editor:,"{'main': 'On This First Day, a Fanfare for the New Era; Knowing the World', 'kicker': None, 'content_kicker': None, 'print_headline': 'On This First Day, a Fanfare for the New Era; Knowing the World', 'name': None, 'seo': None, 'sub': None}",Opinion
2,2000-01-01 05:00:00+00:00,"Many experts on Y2K computer problem report that Internet performed impressively through rollover from 1999 to 2000 even if it did sag at times in isolated sites as users turned to e-mail messages, Web sites, newsgroups and electronic chat rooms to track arrival of Year 2000 in more and more time zones (M)","As the world slid nervously yesterday through the shadow of its first global technology challenge, many year 2000 experts took special comfort in the relatively stable behavior of one of their principal tools in dealing with the problem: the not-always-reliable Internet.","{'main': ""Internet's Cheering Squad Nervously Watches Clock"", 'kicker': '1/1/00: TECHNOLOGY AND 2000 -- MOMENTOUS RELIEF', 'content_kicker': None, 'print_headline': ""1/1/00: TECHNOLOGY AND 2000 -- MOMENTOUS RELIEF; Internet's Cheering Squad Nervously Watches Clock"", 'name': None, 'seo': None, 'sub': None}",U.S.
3,2000-01-01 05:00:00+00:00,"WILL the forces of globalism continue to push the world toward American-style capitalism? As the 21st century begins, advocates of the free market have no doubt that they have won the economic argument. Socialism is dead. Moreover, as a means of creating wealth and material progress, American capitalism seems to be clearly superior to the Asian variety, with its greater level of government planning, or the European version, with its emphasis on social welfare and protection of workers from losing their jobs.",WILL the forces of globalism continue to push the world toward American-style capitalism?,"{'main': 'Economic Thinking Finds a Free Market', 'kicker': 'VISIONS: POWER', 'content_kicker': None, 'print_headline': 'VISIONS: POWER; Economic Thinking Finds a Free Market', 'name': None, 'seo': None, 'sub': None}",Archives
4,2000-01-01 05:00:00+00:00,"SPECIAL TODAY The Millennium Envisioning the future of politics, people, cities and machines -- even the front page of The Times -- and reflecting on the past. SECTION E",SPECIAL TODAY,"{'main': 'INSIDE', 'kicker': None, 'content_kicker': None, 'print_headline': 'INSIDE', 'name': None, 'seo': None, 'sub': None}",New York


In [9]:
#Get just with main sector of column headline
import re

pattern = r"'main'\s*:\s*'((?:[^'\\]|\\.)*)'|" + \
           r"'main'\s*:\s*\"((?:[^\"\\]|\\.)*)\""

clean_nyt_df['headline_clean'] = nyt_df['headline'].str.extract(pattern, flags=re.IGNORECASE)[0].fillna(
    nyt_df['headline'].str.extract(pattern, flags=re.IGNORECASE)[1]
)

In [10]:
clean_nyt_df['all_text'] = (clean_nyt_df['headline_clean'] + 
                              " " + clean_nyt_df['lead_paragraph'] + 
                              " " + clean_nyt_df['abstract']
                             ).str.strip()

clean_nyt_df.head(3)

,date,abstract,lead_paragraph,headline,section_name,headline_clean,all_text
0,2000-01-01 05:00:00+00:00,Article on upcoming New York Giants-Dallas Cowboys game; photo (M),"Waiting in the visiting locker room at Texas Stadium late tomorrow afternoon, the Giants will know whether the Green Bay Packers, who play earlier against Arizona, have won or are comfortably ahead.","{'main': 'Playoffs or No, Dallas Provides The Motivation', 'kicker': 'PRO FOOTBALL', 'content_kicker': None, 'print_headline': 'PRO FOOTBALL; Playoffs or No, Dallas Provides The Motivation', 'name': None, 'seo': None, 'sub': None}",Sports,"Playoffs or No, Dallas Provides The Motivation","Playoffs or No, Dallas Provides The Motivation Waiting in the visiting locker room at Texas Stadium late tomorrow afternoon, the Giants will know whether the Green Bay Packers, who play earlier against Arizona, have won or are comfortably ahead. Article on upcoming New York Giants-Dallas Cowboys game; photo (M)"
1,2000-01-01 05:00:00+00:00,"Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing",To the Editor:,"{'main': 'On This First Day, a Fanfare for the New Era; Knowing the World', 'kicker': None, 'content_kicker': None, 'print_headline': 'On This First Day, a Fanfare for the New Era; Knowing the World', 'name': None, 'seo': None, 'sub': None}",Opinion,"On This First Day, a Fanfare for the New Era; Knowing the World","On This First Day, a Fanfare for the New Era; Knowing the World To the Editor: Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing"
2,2000-01-01 05:00:00+00:00,"Many experts on Y2K computer problem report that Internet performed impressively through rollover from 1999 to 2000 even if it did sag at times in isolated sites as users turned to e-mail messages, Web sites, newsgroups and electronic chat rooms to track arrival of Year 2000 in more and more time zones (M)","As the world slid nervously yesterday through the shadow of its first global technology challenge, many year 2000 experts took special comfort in the relatively stable behavior of one of their principal tools in dealing with the problem: the not-always-reliable Internet.","{'main': ""Internet's Cheering Squad Nervously Watches Clock"", 'kicker': '1/1/00: TECHNOLOGY AND 2000 -- MOMENTOUS RELIEF', 'content_kicker': None, 'print_headline': ""1/1/00: TECHNOLOGY AND 2000 -- MOMENTOUS RELIEF; Internet's Cheering Squad Nervously Watches Clock"", 'name': None, 'seo': None, 'sub': None}",U.S.,Internet's Cheering Squad Nervously Watches Clock,"Internet's Cheering Squad Nervously Watches Clock As the world slid nervously yesterday through the shadow of its first global technology challenge, many year 2000 experts took special comfort in the relatively stable behavior of one of their principal tools in dealing with the problem: the not-always-reliable Internet. Many experts on Y2K computer problem report that Internet performed impressively through rollover from 1999 to 2000 even if it did sag at times in isolated sites as users turned to e-mail messages, Web sites, newsgroups and electronic chat rooms to track arrival of Year 2000 in more and more time zones (M)"


In [11]:
clean_nyt_df['date'] = clean_nyt_df['date'].apply(
                            lambda x: str(x).split()[0] if pd.notnull(x) else None
                        )

clean_nyt_df.head(3)

,date,abstract,lead_paragraph,headline,section_name,headline_clean,all_text
0,2000-01-01,Article on upcoming New York Giants-Dallas Cowboys game; photo (M),"Waiting in the visiting locker room at Texas Stadium late tomorrow afternoon, the Giants will know whether the Green Bay Packers, who play earlier against Arizona, have won or are comfortably ahead.","{'main': 'Playoffs or No, Dallas Provides The Motivation', 'kicker': 'PRO FOOTBALL', 'content_kicker': None, 'print_headline': 'PRO FOOTBALL; Playoffs or No, Dallas Provides The Motivation', 'name': None, 'seo': None, 'sub': None}",Sports,"Playoffs or No, Dallas Provides The Motivation","Playoffs or No, Dallas Provides The Motivation Waiting in the visiting locker room at Texas Stadium late tomorrow afternoon, the Giants will know whether the Green Bay Packers, who play earlier against Arizona, have won or are comfortably ahead. Article on upcoming New York Giants-Dallas Cowboys game; photo (M)"
1,2000-01-01,"Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing",To the Editor:,"{'main': 'On This First Day, a Fanfare for the New Era; Knowing the World', 'kicker': None, 'content_kicker': None, 'print_headline': 'On This First Day, a Fanfare for the New Era; Knowing the World', 'name': None, 'seo': None, 'sub': None}",Opinion,"On This First Day, a Fanfare for the New Era; Knowing the World","On This First Day, a Fanfare for the New Era; Knowing the World To the Editor: Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing"
2,2000-01-01,"Many experts on Y2K computer problem report that Internet performed impressively through rollover from 1999 to 2000 even if it did sag at times in isolated sites as users turned to e-mail messages, Web sites, newsgroups and electronic chat rooms to track arrival of Year 2000 in more and more time zones (M)","As the world slid nervously yesterday through the shadow of its first global technology challenge, many year 2000 experts took special comfort in the relatively stable behavior of one of their principal tools in dealing with the problem: the not-always-reliable Internet.","{'main': ""Internet's Cheering Squad Nervously Watches Clock"", 'kicker': '1/1/00: TECHNOLOGY AND 2000 -- MOMENTOUS RELIEF', 'content_kicker': None, 'print_headline': ""1/1/00: TECHNOLOGY AND 2000 -- MOMENTOUS RELIEF; Internet's Cheering Squad Nervously Watches Clock"", 'name': None, 'seo': None, 'sub': None}",U.S.,Internet's Cheering Squad Nervously Watches Clock,"Internet's Cheering Squad Nervously Watches Clock As the world slid nervously yesterday through the shadow of its first global technology challenge, many year 2000 experts took special comfort in the relatively stable behavior of one of their principal tools in dealing with the problem: the not-always-reliable Internet. Many experts on Y2K computer problem report that Internet performed impressively through rollover from 1999 to 2000 even if it did sag at times in isolated sites as users turned to e-mail messages, Web sites, newsgroups and electronic chat rooms to track arrival of Year 2000 in more and more time zones (M)"


In [12]:
#Get just relevant columns
nyt_news = clean_nyt_df[['date', 'all_text']].copy()
nyt_news.head()

,date,all_text
0,2000-01-01,"Playoffs or No, Dallas Provides The Motivation Waiting in the visiting locker room at Texas Stadium late tomorrow afternoon, the Giants will know whether the Green Bay Packers, who play earlier against Arizona, have won or are comfortably ahead. Article on upcoming New York Giants-Dallas Cowboys game; photo (M)"
1,2000-01-01,"On This First Day, a Fanfare for the New Era; Knowing the World To the Editor: Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing"
2,2000-01-01,"Internet's Cheering Squad Nervously Watches Clock As the world slid nervously yesterday through the shadow of its first global technology challenge, many year 2000 experts took special comfort in the relatively stable behavior of one of their principal tools in dealing with the problem: the not-always-reliable Internet. Many experts on Y2K computer problem report that Internet performed impressively through rollover from 1999 to 2000 even if it did sag at times in isolated sites as users turned to e-mail messages, Web sites, newsgroups and electronic chat rooms to track arrival of Year 2000 in more and more time zones (M)"
3,2000-01-01,"Economic Thinking Finds a Free Market WILL the forces of globalism continue to push the world toward American-style capitalism? WILL the forces of globalism continue to push the world toward American-style capitalism? As the 21st century begins, advocates of the free market have no doubt that they have won the economic argument. Socialism is dead. Moreover, as a means of creating wealth and material progress, American capitalism seems to be clearly superior to the Asian variety, with its greater level of government planning, or the European version, with its emphasis on social welfare and protection of workers from losing their jobs."
4,2000-01-01,"INSIDE SPECIAL TODAY SPECIAL TODAY The Millennium Envisioning the future of politics, people, cities and machines -- even the front page of The Times -- and reflecting on the past. SECTION E"


In [13]:
#Get lowercase
nyt_news['clean_text'] = nyt_news['all_text'].str.lower()
nyt_news.head(2)

,date,all_text,clean_text
0,2000-01-01,"Playoffs or No, Dallas Provides The Motivation Waiting in the visiting locker room at Texas Stadium late tomorrow afternoon, the Giants will know whether the Green Bay Packers, who play earlier against Arizona, have won or are comfortably ahead. Article on upcoming New York Giants-Dallas Cowboys game; photo (M)","playoffs or no, dallas provides the motivation waiting in the visiting locker room at texas stadium late tomorrow afternoon, the giants will know whether the green bay packers, who play earlier against arizona, have won or are comfortably ahead. article on upcoming new york giants-dallas cowboys game; photo (m)"
1,2000-01-01,"On This First Day, a Fanfare for the New Era; Knowing the World To the Editor: Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing","on this first day, a fanfare for the new era; knowing the world to the editor: jeanne c pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing"


In [14]:
#Remove special characters
nyt_news['clean_text'] = nyt_news['clean_text'].str.replace(r'[^\w]', ' ', regex=True)
nyt_news.head(2)

,date,all_text,clean_text
0,2000-01-01,"Playoffs or No, Dallas Provides The Motivation Waiting in the visiting locker room at Texas Stadium late tomorrow afternoon, the Giants will know whether the Green Bay Packers, who play earlier against Arizona, have won or are comfortably ahead. Article on upcoming New York Giants-Dallas Cowboys game; photo (M)",playoffs or no dallas provides the motivation waiting in the visiting locker room at texas stadium late tomorrow afternoon the giants will know whether the green bay packers who play earlier against arizona have won or are comfortably ahead article on upcoming new york giants dallas cowboys game photo m
1,2000-01-01,"On This First Day, a Fanfare for the New Era; Knowing the World To the Editor: Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing",on this first day a fanfare for the new era knowing the world to the editor jeanne c pond letter expresses hope that spiritual development artistic knowledge and skills and self esteem flourish in new century drawing


In [15]:
nyt_news['section_name'] = nyt_df['section_name']

In [16]:
nyt_news.head()

,date,all_text,clean_text,section_name
0,2000-01-01,"Playoffs or No, Dallas Provides The Motivation Waiting in the visiting locker room at Texas Stadium late tomorrow afternoon, the Giants will know whether the Green Bay Packers, who play earlier against Arizona, have won or are comfortably ahead. Article on upcoming New York Giants-Dallas Cowboys game; photo (M)",playoffs or no dallas provides the motivation waiting in the visiting locker room at texas stadium late tomorrow afternoon the giants will know whether the green bay packers who play earlier against arizona have won or are comfortably ahead article on upcoming new york giants dallas cowboys game photo m,Sports
1,2000-01-01,"On This First Day, a Fanfare for the New Era; Knowing the World To the Editor: Jeanne C Pond letter expresses hope that spiritual development, artistic knowledge and skills and self-esteem flourish in new century; drawing",on this first day a fanfare for the new era knowing the world to the editor jeanne c pond letter expresses hope that spiritual development artistic knowledge and skills and self esteem flourish in new century drawing,Opinion
2,2000-01-01,"Internet's Cheering Squad Nervously Watches Clock As the world slid nervously yesterday through the shadow of its first global technology challenge, many year 2000 experts took special comfort in the relatively stable behavior of one of their principal tools in dealing with the problem: the not-always-reliable Internet. Many experts on Y2K computer problem report that Internet performed impressively through rollover from 1999 to 2000 even if it did sag at times in isolated sites as users turned to e-mail messages, Web sites, newsgroups and electronic chat rooms to track arrival of Year 2000 in more and more time zones (M)",internet s cheering squad nervously watches clock as the world slid nervously yesterday through the shadow of its first global technology challenge many year 2000 experts took special comfort in the relatively stable behavior of one of their principal tools in dealing with the problem the not always reliable internet many experts on y2k computer problem report that internet performed impressively through rollover from 1999 to 2000 even if it did sag at times in isolated sites as users turned to e mail messages web sites newsgroups and electronic chat rooms to track arrival of year 2000 in more and more time zones m,U.S.
3,2000-01-01,"Economic Thinking Finds a Free Market WILL the forces of globalism continue to push the world toward American-style capitalism? WILL the forces of globalism continue to push the world toward American-style capitalism? As the 21st century begins, advocates of the free market have no doubt that they have won the economic argument. Socialism is dead. Moreover, as a means of creating wealth and material progress, American capitalism seems to be clearly superior to the Asian variety, with its greater level of government planning, or the European version, with its emphasis on social welfare and protection of workers from losing their jobs.",economic thinking finds a free market will the forces of globalism continue to push the world toward american style capitalism will the forces of globalism continue to push the world toward american style capitalism as the 21st century begins advocates of the free market have no doubt that they have won the economic argument socialism is dead moreover as a means of creating wealth and material progress american capitalism seems to be clearly superior to the asian variety with its greater level of government planning or the european version with its emphasis on social welfare and protection of workers from losing their jobs,Archives
4,2000-01-01,"INSIDE SPECIAL TODAY SPECIAL TODAY The Millennium Envisioning the future of politics, people, cities and machines -- even the front page of The Times -- and reflecting on the past. SECTION E",inside special today special today the millennium envisioning the future of politic

In [19]:
business_news = nyt_news[nyt_news['section_name'] == 'Business Day']

In [20]:
business_news.shape

(273577, 4)

In [24]:
keywords = ["SP 500", "S P 500""S&P 500", "SPY", "SPX", "GSPC", "S&P500", "Standard & Poor's 500", "500 Index", "VOO", "IVV"]
keywords = [keyword.lower() for keyword in keywords]

SPY_news = business_news[business_news["clean_text"].str.contains('|'.join(keywords), case=False, na=False)]


In [25]:
SPY_news.head()

,date,all_text,clean_text,section_name
12964,2000-02-17,"Exchange to Offer Derivatives Linked to the Fortune 500 List Move over Dow Jones and Standard & Poor's, here comes the Fortune 500 stock index. Fortune magazine says it is going to allow Chicago Mercantile Exchange to begain offering futures and options linked to new Fortune 500 index, which magazine will create; magazine says it will also license exchange to trade derivatives based on newly developed Fortune E-50 index, which would track performance of companies tied to Internet economy (M)",exchange to offer derivatives linked to the fortune 500 list move over dow jones and standard poor s here comes the fortune 500 stock index fortune magazine says it is going to allow chicago mercantile exchange to begain offering futures and options linked to new fortune 500 index which magazine will create magazine says it will also license exchange to trade derivatives based on newly developed fortune e 50 index which would track performance of companies tied to internet economy m,Business Day
15427,2000-02-26,"STOCKS IN TURMOIL AS WORRIES GROW ON HIGHER RATES The Dow Jones industrial average, which reached a peak of 11,723 last month, closed below the 10,000 mark yesterday for the first time in 10 months as investors dumped shares of companies most vulnerable to higher interest rates. The other popular stock indexes followed suit. Dow Jones industrial average, which reached peak of 11,723 in January, closes below 10,000 for first time in 10 months as investors dump shares of companies most vulnerable to higher interest rates; declines 230.51 points to 9,862.12, down 3.5 percent for week on New York Stock Exchange; other indexes follow suit; Nasdaq falls 27.15 points, to 4,590.50, and Standard & Poor's 500 index 20.07, to 1,333.36; graphs (M)",stocks in turmoil as worries grow on higher rates the dow jones industrial average which reached a peak of 11 723 last month closed below the 10 000 mark yesterday for the first time in 10 months as investors dumped shares of companies most vulnerable to higher interest rates the other popular stock indexes followed suit dow jones industrial average which reached peak of 11 723 in january closes below 10 000 for first time in 10 months as investors dump shares of companies most vulnerable to higher interest rates declines 230 51 points to 9 862 12 down 3 5 percent for week on new york stock exchange other indexes follow suit nasdaq falls 27 15 points to 4 590 50 and standard poor s 500 index 20 07 to 1 333 36 graphs m,Business Day
15777,2000-02-27,"In Technology, Less Than Meets the Eye DON'T fight the Fed. That is one of Wall Street's must venerable and trusted dictums. Gretchen Morgenson (Market Watch) column says that while Nasdaq composite index appears to be strong, up 4 percent this week to 4,590.50, there is less to its move than meets the eye; notes that according to Credit Suisse First Boston, only 34 percent of e-commerce stocks are up this year and computer retailers are down 9 percent; says as result, technology as whole is up just 2 percent; graph on Nasdaq and S&P 500 indexes (M)",in technology less than meets the eye don t fight the fed that is one of wall street s must venerable and trusted dictums gretchen morgenson market watch column says that while nasdaq composite index appears to be strong up 4 percent this week to 4 590 50 there is less to its move than meets the eye notes that according to credit suisse first boston only 34 percent of e commerce stocks are up this year and computer retailers are down 9 percent says as result technology as whole is up just 2 percent graph on nasdaq and s p 500 indexes m,Business Day
21892,2000-03-20,"Media Talk; Putin's Memoirs Set for Publication in U.S. PublicAffairs, a New York company that is fast becoming the publishing house for Kremlin politicians, is preparing to rush into print in May with an English translation of the memoirs describing the life, travails and cold war spying 

In [26]:
SPY_news.shape

(620, 4)

In [27]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [28]:
def token_lemma_nonstops(text):
    doc = nlp(text)
    output = [token.lemma_ for token in doc if not token.is_stop]
    output = ' '.join(output)
    return output

In [29]:
SPY_news['clean_text'] = SPY_news['clean_text'].astype(str)

/var/folders/gt/_n6gk0zn66v7tnqb9xpcyk_c0000gn/T/ipykernel_88429/1609007845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPY_news['clean_text'] = SPY_news['clean_text'].astype(str)


In [30]:
SPY_news['clean_text'] = SPY_news['clean_text'].apply(token_lemma_nonstops)

/var/folders/gt/_n6gk0zn66v7tnqb9xpcyk_c0000gn/T/ipykernel_88429/3021595863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SPY_news['clean_text'] = SPY_news['clean_text'].apply(token_lemma_nonstops)


In [31]:
SPY_news.head()

,date,all_text,clean_text,section_name
12964,2000-02-17,"Exchange to Offer Derivatives Linked to the Fortune 500 List Move over Dow Jones and Standard & Poor's, here comes the Fortune 500 stock index. Fortune magazine says it is going to allow Chicago Mercantile Exchange to begain offering futures and options linked to new Fortune 500 index, which magazine will create; magazine says it will also license exchange to trade derivatives based on newly developed Fortune E-50 index, which would track performance of companies tied to Internet economy (M)",exchange offer derivative link fortune 500 list dow jones standard poor s come fortune 500 stock index fortune magazine say go allow chicago mercantile exchange begain offer future option link new fortune 500 index magazine create magazine say license exchange trade derivative base newly develop fortune e 50 index track performance company tie internet economy m,Business Day
15427,2000-02-26,"STOCKS IN TURMOIL AS WORRIES GROW ON HIGHER RATES The Dow Jones industrial average, which reached a peak of 11,723 last month, closed below the 10,000 mark yesterday for the first time in 10 months as investors dumped shares of companies most vulnerable to higher interest rates. The other popular stock indexes followed suit. Dow Jones industrial average, which reached peak of 11,723 in January, closes below 10,000 for first time in 10 months as investors dump shares of companies most vulnerable to higher interest rates; declines 230.51 points to 9,862.12, down 3.5 percent for week on New York Stock Exchange; other indexes follow suit; Nasdaq falls 27.15 points, to 4,590.50, and Standard & Poor's 500 index 20.07, to 1,333.36; graphs (M)",stock turmoil worry grow high rate dow jones industrial average reach peak 11 723 month close 10 000 mark yesterday time 10 month investor dump share company vulnerable high interest rate popular stock index follow suit dow jones industrial average reach peak 11 723 january close 10 000 time 10 month investor dump share company vulnerable high interest rate decline 230 51 point 9 862 12 3 5 percent week new york stock exchange index follow suit nasdaq fall 27 15 point 4 590 50 standard poor s 500 index 20 07 1 333 36 graph m,Business Day
15777,2000-02-27,"In Technology, Less Than Meets the Eye DON'T fight the Fed. That is one of Wall Street's must venerable and trusted dictums. Gretchen Morgenson (Market Watch) column says that while Nasdaq composite index appears to be strong, up 4 percent this week to 4,590.50, there is less to its move than meets the eye; notes that according to Credit Suisse First Boston, only 34 percent of e-commerce stocks are up this year and computer retailers are down 9 percent; says as result, technology as whole is up just 2 percent; graph on Nasdaq and S&P 500 indexes (M)",technology meet eye don t fight fed wall street s venerable trust dictum gretchen morgenson market watch column say nasdaq composite index appear strong 4 percent week 4 590 50 meet eye note accord credit suisse boston 34 percent e commerce stock year computer retailer 9 percent say result technology 2 percent graph nasdaq s p 500 index m,Business Day
21892,2000-03-20,"Media Talk; Putin's Memoirs Set for Publication in U.S. PublicAffairs, a New York company that is fast becoming the publishing house for Kremlin politicians, is preparing to rush into print in May with an English translation of the memoirs describing the life, travails and cold war spying adventures of Russia's acting president, Vladimir V. Putin. PublicAffairs plans to publish memoirs of Russia's acting president, Vladimir V Putin; original Russian version of First Person is still unavailable in Russia, where publication is barred until after Mar 26 presidential election; photo (M)",medium talk putin s memoir set publication u s publicaffair new york company fast publishing house kremlin politician prepare rush print english translation memoir describe life travail cold war spy adventure russia s ac

In [32]:
SPY_news.to_csv('../Data/Preprocessed/SPY-news.csv')